# Coins Table
#### Stores the cryptocurrency names, symbols & slugs as well as coin_id (PK).

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [196]:
#Coins Table : top 100 currencies names/symbols/slugs
df_names = pd.read_csv('/Users/nazihkalo/Desktop/UChicago_Analytics/Quarter 1/DEPA/Final Project/Data/crypto_names.csv')
df_names.head()

,coin_id,name,symbol,slug
0,0,0x,ZRX,0x
1,1,Aeternity,AE,aeternity
2,2,Aion,AION,aion
3,3,Ardor,ARDR,ardor
4,4,Ark,ARK,ark


# Reddit Table
#### Reddit posts from /r/Cryptocurrency from 2017-2019

In [5]:
import praw
from psaw import PushshiftAPI

In [6]:
#Connect to reddit API
reddit = praw.Reddit(client_id='GjfUHQE8AYnXLg', client_secret='PfbhtsXJGAAUNiEyHPRGPuFJ0ro', user_agent='DEPA_Project')
api = PushshiftAPI()

In [7]:
#Extract all posts to /r/CryptoCurrency from 2017-2019

start_epoch=int(dt.datetime(2017, 1, 1).timestamp())
end_epoch = int(dt.datetime(2019, 1, 1).timestamp())

reddit_data = pd.DataFrame(api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit='CryptoCurrency',
                            filter=['author', 'title', 'subreddit', 'num_comments', 'created', 'score']))

/anaconda3/lib/python3.7/site-packages/psaw/PushshiftAPI.py:153: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [8]:
#Convert the created_utc column to a date 
reddit_data['created_utc']=(pd.to_datetime(reddit_data['created_utc'],unit='s'))

In [ ]:
#remove redundant columns
reddit_data = reddit_data.drop(['created', 'd_'], axis = 1)

In [11]:
#Rename index to post_id
reddit_data.index.names = ['post_id']

In [76]:
#convert created_utc to just a date
reddit_data['created_utc'] = reddit_data['created_utc'].dt.date

In [77]:
reddit_data.head()

,author,created_utc,num_comments,score,subreddit,title
post_id,,,,,,
0,robertbint,2019-01-01,0,1,CryptoCurrency,Buy or Sell Bitcoins online - A Few Pointers a...
1,h214289,2019-01-01,0,1,CryptoCurrency,Dec 31 2018 Important Crypto News
2,coinmarshal,2019-01-01,5,1,CryptoCurrency,I am sharing Crypto with my WhatsApp buddies. ...
3,h1121900,2019-01-01,0,1,CryptoCurrency,Dec 31 2018 Important Crypto News
4,instasmarter,2019-01-01,13,1,CryptoCurrency,Best places to spend Bitcoin


# Create reddit_coins Join Table 
#### Matches the reddit posts in reddit_data table to the coins in coins table

In [13]:
#Zip the names into tuple of (name, slug, symbol)
zipped_names = list(zip(df_names['name'],df_names['slug'],df_names['symbol']))

#Create a search list - seperating the three terms with OR operator (|)
search_list = []
for (name, slug, symbol) in zipped_names:
    listt = [name, slug, symbol]
    pat = '|'.join(listt)
    search_list.append(pat)

In [15]:
#Add boolean columns
import re
dummy_df = pd.DataFrame(dict((name, reddit_data.title.str.contains(name, re.IGNORECASE))
                             for name in search_list))

In [16]:
dummy_df.head()

,0x|0x|ZRX,Aeternity|aeternity|AE,Aion|aion|AION,Ardor|ardor|ARDR,Ark|ark|ARK,Augur|augur|REP,Aurora|aurora|AOA,Bancor|bancor|BNT,Basic Attention Token|basic-attention-token|BAT,Binance Coin|binance-coin|BNB,...,VeChain|vechain|VET,Verge|verge|XVG,WAX|wax|WAX,Waltonchain|waltonchain|WTC,Wanchain|wanchain|WAN,Waves|waves|WAVES,XRP|ripple|XRP,Zcash|zcash|ZEC,Zilliqa|zilliqa|ZIL,aelf|aelf|ELF
post_id,,,,,,,,,,,,,,,,,,,,,
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
#Convert dummy columns into rows with post_id as the index
i, j = np.where(dummy_df)

coins_mentioned_series = pd.Series(dict(zip(zip(i, j), dummy_df.columns[j])))

In [30]:
#Create final join table between reddit and coins table. Rename columns of dataframe. 
coin_reddit_join = pd.DataFrame(coins_mentioned_series).reset_index()
coin_reddit_join.columns = ['post_id', 'coin_id', 'coin_name']
coin_reddit_join = coin_reddit_join.drop('coin_name', axis = 1)
coin_reddit_join.head()

,post_id,coin_id
0,0,11
1,4,11
2,7,11
3,8,4
4,8,35


# Pricing Table

In [55]:
import requests
import datetime
import pandas as pd

In [56]:
def daily_price_historical(symbol, comparison_symbol, all_data=True, limit=1, aggregate=1, exchange=''):
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
    return df

In [57]:
pricing_list = []

for symbol in list(df_names['symbol']):
# initialise scraper with time interval
    try:
        df = daily_price_historical(symbol, 'USD')
        df['name'] = symbol
        pricing_list.append(df)
    except:
        df2 = pd.DataFrame(columns = df.columns)
        pricing_list.append(df2)

In [59]:
pricing_df = pd.concat(pricing_list)

In [65]:
pricing_df.head()

,close,high,low,open,time,volumefrom,volumeto,timestamp,name
0,0.06687,0.4444,0.06687,0.06950,1502409600,39437.24,2994.43,2017-08-10 19:00:00,ZRX
1,0.16250,0.1830,0.06687,0.06687,1502496000,6232.81,765.38,2017-08-11 19:00:00,ZRX
2,0.19000,0.3880,0.12500,0.16250,1502582400,10382.08,1885.11,2017-08-12 19:00:00,ZRX
3,0.32000,0.3300,0.13100,0.19000,1502668800,7936.82,1788.88,2017-08-13 19:00:00,ZRX
4,1.00000,1.8000,0.32000,0.32000,1502755200,3467.80,2905.58,2017-08-14 19:00:00,ZRX


In [79]:
pricing_df2 = pricing_df.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'name', right_on = 'symbol')
pricing_df3 = pricing_df2.drop(['name', 'symbol', 'time'], axis = 1)

In [80]:
pricing_df3['timestamp'] = pricing_df3['timestamp'].dt.date

In [81]:
pricing_df3.head()

,close,high,low,open,volumefrom,volumeto,timestamp,coin_id
0,0.06687,0.4444,0.06687,0.06950,39437.24,2994.43,2017-08-10,0
1,0.16250,0.1830,0.06687,0.06687,6232.81,765.38,2017-08-11,0
2,0.19000,0.3880,0.12500,0.16250,10382.08,1885.11,2017-08-12,0
3,0.32000,0.3300,0.13100,0.19000,7936.82,1788.88,2017-08-13,0
4,1.00000,1.8000,0.32000,0.32000,3467.80,2905.58,2017-08-14,0


# Google Trends table

In [78]:
#Pulling google trends data from cryptory 

i=1
google_data_list = []
for name in list(df_names['name']):
    if(i>0 and i<101):
        i=i+1
        kw_list = []
        kw_list.append(name)
        try:
            data = my_cryptory.get_google_trends(kw_list)
            google_data_list.append(data)
        except:
            continue;

/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


In [111]:
trend_df1 = pd.concat(google_data_list)

trend_df2 = pd.DataFrame(df1.pivot_table(index = 'date').unstack()).reset_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [118]:
#Adding coin_id column
trend_df3 = trend_df2.merge(df_names[['name', 'coin_id']], how = 'left', left_on = 'level_0', right_on = 'name')

#Dropping columns
trend_df3 = trend_df3.drop(['level_0', 'name'], axis = 1)

#Rename columns
trend_df3.columns = ['date', 'trend', 'coin_id']

In [119]:
trend_df3.head()

,date,trend,coin_id
0,2017-01-01,14.939345,0
1,2017-01-02,14.939345,0
2,2017-01-03,15.686312,0
3,2017-01-04,24.276435,0
4,2017-01-05,22.035534,0


# Twitter Table

In [125]:
import re
import csv
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML

In [182]:
import pandas as pd
tweets_df = pd.DataFrame({'Date': [], 'No. of Tweets': [], 'Coin': []})

for i in range(0,len(df_names['symbol'])-1):
    #len(coin_name["symbol"])-1

    coin = df_names['symbol'][i]
    url = 'https://bitinfocharts.com/comparison/tweets-'+coin.lower()+'.html'
    headers = {'User-Agent': "Chrome/54.0.2840.90"}
    response = requests.get(url, headers=headers)
    html = response.text 

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    x = soup.find_all('script')

    data_1 = re.findall(r'(\[new\sDate.*\]])', str(x))
    data_1 = str(data_1)
    
    
    if data_1 == '[]':
        continue
    data_2 = data_1.split("],[")
    data_2[0] = data_2[0][3:]
    data_2[len(data_2) - 1] = data_2[len(data_2) - 1][:-4]
    data_pd = pd.DataFrame(data_2)
    data_clean = data_pd[0].str.split(",", expand = True)
    data_clean = data_clean.iloc[:, 0:2]
    data_clean.columns = ['Date', 'No. of Tweets']
    data_clean["Date"] = data_clean["Date"].str.slice(10,20)
    data_clean["Coin"] = coin.upper()
    tweets_df = tweets_df.append(data_clean, ignore_index = True)

In [183]:
#Convert Date into datetime object
tweets_df['Date'] = tweets_df['Date'].astype('datetime64')

In [184]:
#Remove the null rows
tweets_df = tweets_df[tweets_df['No. of Tweets'] != 'null']

In [185]:
#Convert No. of Tweets into int object
tweets_df['No. of Tweets'] = tweets_df['No. of Tweets'].astype('int')

In [186]:
#Adding coin_id column
tweets_df = tweets_df.merge(df_names[['symbol', 'coin_id']], how = 'left', left_on = 'Coin', right_on = 'symbol')

#Drop redundant name columns
tweets_df = tweets_df.drop(['Coin', 'symbol'], axis = 1)

In [187]:
tweets_df.head()

,Date,No. of Tweets,coin_id
0,2018-01-08,7,0
1,2018-01-09,20,0
2,2018-01-10,21,0
3,2018-01-11,4,0
4,2018-01-12,6,0


# Reddit Subscribers

In [189]:
# load package
from cryptory import Cryptory

# initialise object 
# pull data from start of 2017 to present day
my_cryptory = Cryptory(from_date = "2017-01-01",to_date="2019-01-01", ascending=True)

In [200]:
subreddit_list = []
for name in list(df_names['symbol']):
    try:
        subred = my_cryptory.extract_reddit_metrics(subreddit = name, metric = "total-subscribers", col_label="", sub_col=True)
        subreddit_list.append(subred)
    except:
        pass

In [204]:
pd.concat(subreddit_list,)

TypeError: cannot concatenate object of type "<class 'ValueError'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

# Saving our tables into CSV files

In [120]:
reddit_data.to_csv(path_or_buf = '/Users/nazihkalo/Desktop/UChicago_Analytics/Quarter 1/DEPA/Final Project/Data/reddit_posts.csv')

In [122]:
coin_reddit_join.to_csv(path_or_buf = '/Users/nazihkalo/Desktop/UChicago_Analytics/Quarter 1/DEPA/Final Project/Data/coin_reddit.csv')

In [123]:
trend_df3.to_csv(path_or_buf = '/Users/nazihkalo/Desktop/UChicago_Analytics/Quarter 1/DEPA/Final Project/Data/google_trends.csv')

In [124]:
pricing_df3.to_csv(path_or_buf = '/Users/nazihkalo/Desktop/UChicago_Analytics/Quarter 1/DEPA/Final Project/Data/pricing.csv')

In [188]:
tweets_df.to_csv(path_or_buf = '/Users/nazihkalo/Desktop/UChicago_Analytics/Quarter 1/DEPA/Final Project/Data/tweets.csv')